- Source: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [123]:
import tableauserverclient as TSC
import pandas as pd
import numpy as np
import pickle as pk

pd.set_option("display.max_rows", None)

In [92]:
hr = pd.read_excel("D:/디지털혁신팀/태블로/직원명단_조직_210630.xlsx")
hr["성명"] = hr["성명"].apply(lambda x:x.replace(" ", ""))
hr = hr[["사번", "본부명(조직단위기준)", "조직명", "하위그룹", "직무", "직급", "성명"]]
hr = hr.rename({"사번":"num", "본부명(조직단위기준)":"hq", "조직명":"dep", "하위그룹":"sub", "직무":"job", "직급":"rank", "성명":"name"}, axis=1)
hr = hr.sort_values(by=["job", "rank", "hq", "dep"], axis=0)
hr["num"] = hr["num"].map(str)

In [93]:
empls = {str(row["num"]):(row["hq"], row["dep"], row["job"], row["name"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

## User Groups

In [104]:
hr.loc[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장"), "check"] = "1그룹"
hr.loc[(hr["job"] == "PM") | (hr["job"] == "건축PM") | (hr["job"] == "팀장") | (hr["job"] == "실장") | (hr["job"] == "그룹장") | (hr["job"] == "지사장") | (hr["num"] == "6447"), "check"] = "2그룹"
hr.loc[(hr["job"] == "소장") | (hr["job"] == "부문장") | (hr["job"] == "리더"), "check"] = "3그룹"

In [72]:
# 1그룹
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장")]

,num,hq,dep,job,rank,name
900,6379,사장직속,대표이사사장,대표이사사장,사장,권순호
114,3904,건설본부,건설본부,본부장,상무,하원기
914,6400,개발영업본부,개발영업본부,본부장,전무,박희윤
921,6415,경영본부,경영본부,본부장,전무,정경구
959,6459,미래혁신본부,미래혁신본부,본부장,전무,김홍일
899,6377,부회장직속,부회장,부회장,부회장,김대철
0,1,회장직속,회장,회장,회장,정몽규


In [75]:
# 2그룹
hr[(hr["job"] == "PM") | (hr["job"] == "건축PM") | (hr["job"] == "팀장") | (hr["job"] == "실장") | (hr["job"] == "그룹장") | (hr["job"] == "지사장") | (hr["num"] == "6447")]

,num,hq,dep,job,rank,name
115,3911,건설본부,인프라PM,PM,1급,김범동
427,5421,건설본부,인프라PM,PM,1급,이종민
949,6447,경영본부,경영혁신TFT,TFT팀장,전무,이형기
11,2912,건설본부,건축PM,건축PM,1급,김동진
20,3109,건설본부,건축PM,건축PM,상무,배영선
86,3755,건설본부,건축PM,건축PM,상무,이창규
106,3817,건설본부,건축PM,건축PM,상무,이현대
166,4187,개발영업본부,설계그룹(개발영업),그룹장,1급,박찬호
358,5011,개발영업본부,견적그룹(개발영업),그룹장,2급,소문석
561,5820,건설본부,설비그룹(건설),그룹장,2급,현규섭


In [78]:
# 3그룹
hr[(hr["job"] == "소장") | (hr["job"] == "부문장") | (hr["job"] == "리더")]

,num,hq,dep,job,rank,name
944,6442,개발영업본부,마케팅부문,부문장,1급,이정은
819,6253,개발영업본부,개발부문,부문장,상무,김태수
953,6453,개발영업본부,수주부문,부문장,상무,김대중
422,5381,개발영업본부,인프라부문,부문장,상무,박상일
78,3684,건설본부,TheH Firstier IPARK(개포1 주택재건축),소장,1급,이종일
132,4009,건설본부,둔촌주공재건축,소장,1급,김용남
231,4540,건설본부,시티오씨엘3단지,소장,1급,양승철
288,4826,건설본부,아산병원P동및I동증축공사,소장,1급,민성우
111,3872,건설본부,평택고덕IPARK,소장,1급,김창범
135,4057,건설본부,DMC SK뷰 아이파크포레(수색13구역재개발),소장,2급,안정찬


## SIgn In

In [94]:
with open("D:/디지털혁신팀/태블로/admin_TOKEN.pkl", "rb") as f:
    token = pk.load(f)
tableau_auth = TSC.PersonalAccessTokenAuth("admin_TOKEN", token)
server = TSC.Server("http://218.153.56.75/", use_server_version=True)

## all_users.csv

In [95]:
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name"])
# all_users = all_users.sort_values(by=["hq", "dep", "job", "name", "site_role"])
# all_users = all_users.reset_index(drop=True)

In [96]:
all_users.to_csv("D:/디지털혁신팀/태블로/all_users.csv", index=False, encoding="euc-kr")
all_users.head()

,user_id,user_name,user_fullname,site_role,hq,dep,job,name
0,bdebde0b-b41d-4e9a-a6b7-423ab68bb16b,admin,admin,ServerAdministrator,None,None,None,None
1,a6e879b8-950e-4b05-9ff6-fe5565da5018,viewer,viewer(디지털플랫폼팀),Viewer,None,None,None,None
2,2c2ceaa8-9387-4edb-8ab2-38b17f686304,2912,김동진,Viewer,건설본부,건축PM,건축PM,김동진
3,c232b3be-3aac-4f4f-83fb-bc8a15dab963,3109,배영선,Viewer,건설본부,건축PM,건축PM,배영선
4,e17666e4-a736-40a7-badc-865a60ebd7f2,3139,서태철,Viewer,건설본부,광주학동4구역,소장,서태철


## user_group.csv

In [19]:
user_group = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_groups(user, req_options=req_opts)
        user_group.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (group.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, group.name) for group in user.groups if group.name != "All Users"])
        
user_group = pd.DataFrame(user_group, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name", "group"])
# user_group = user_group.sort_values(by=["hq", "dep", "job", "name", "site_role", "group"])
# user_group = user_group.reset_index(drop=True)

In [20]:
user_group.to_csv("D:/디지털혁신팀/태블로/user_group.csv", index=False, encoding="euc-kr")
user_group.head()

,user_id,user_name,user_fullname,site_role,hq,dep,job,name,group
0,bdebde0b-b41d-4e9a-a6b7-423ab68bb16b,admin,admin,ServerAdministrator,None,None,None,None,디지털혁신팀
1,a6e879b8-950e-4b05-9ff6-fe5565da5018,viewer,viewer(디지털플랫폼팀),Viewer,None,None,None,None,디지털혁신팀
2,2c2ceaa8-9387-4edb-8ab2-38b17f686304,2912,김동진,Viewer,건설본부,건축PM,건축PM,김동진,본사팀장/그룹장
3,c232b3be-3aac-4f4f-83fb-bc8a15dab963,3109,배영선,Viewer,건설본부,건축PM,건축PM,배영선,본사팀장/그룹장
4,123c4e57-2fc6-4a62-8837-d3c1277ee1c6,3449,정상민,Viewer,건설본부,외주구매팀,팀장,정상민,본사팀장/그룹장


## Remove user

In [26]:
user_id = "371ace7c-8abb-484e-9c61-ac1ab2247f4d"
with server.auth.sign_in(tableau_auth):
    server.users.remove(user_id)

## Modify User Information

In [32]:
# for _, row in all_users.iterrows():
#     if row["dep"] != None:
#         print(row["user_id"], row["user_fullname"], row["name"], row["dep"])

2c2ceaa8-9387-4edb-8ab2-38b17f686304 김동진 김동진 건축PM
c232b3be-3aac-4f4f-83fb-bc8a15dab963 배영선 배영선 건축PM
e17666e4-a736-40a7-badc-865a60ebd7f2 서태철 서태철 광주학동4구역
cf30b80d-8e3a-4128-995a-acbe79a7090f 이훈석 이훈석 건설기획팀(신규준비)
f0a544b2-1f42-4f3d-b20f-bcd428b11d6a 김행대 김행대 서울숲IPARK리버포레
123c4e57-2fc6-4a62-8837-d3c1277ee1c6 정상민 정상민 외주구매팀
af48c1c7-5d45-4e6a-bd52-e028d35013cf 김영도 김영도 현대엘리베이터충주신공장
55a1b5a0-fe20-4711-85d4-514c0d1aaec4 김유겸 김유겸 고속국도제32호선아산-천안간건설공사(4공구)
06f0c545-3f70-47ac-a6e3-0fd195c4ea68 이문경 이문경 부산동래래미안아이파크
15d539bf-721c-42f7-a3bb-a0b6cecbf3eb 이종일 이종일 TheH Firstier IPARK(개포1 주택재건축)
ab1b3edd-f03c-4358-baf2-695f7bab45f5 고영진 고영진 고속국도제32호선아산-천안간건설공사(5공구)
102a7f7f-6fa3-41b2-ab18-8fb2357fa3d5 이창규 이창규 건축PM
7f57aeae-354b-4ed7-acf0-fcb7a29ef108 김달운 김달운 아시아드레이카운티(부산거제2구역)
2bc4b236-5eb8-4b69-a32e-5ef51a167d5a 한희태 한희태 김포풍무2지구도시개발사업
9d798eca-8cfe-461f-9a23-5e654cc799c7 김한덕 김한덕 수원센트럴아이파크자이(수원팔달10구역재개발)
4b68efef-e3cc-4820-af1e-54e23bcd5cda 최영준 최영준 아현2구역재건축
693792b3-5945-468a-8652-22479d429b45 김창환 김창환 용산병원부지개발

In [ ]:
# with server.auth.sign_in(tableau_auth):
#     user = server.users.get_by_id(row["user_id"])
#     user.fullname = f"{row['name']}({row['dep']})"
    
#     user_rev = server.users.update(user)

## have_account

In [ ]:
hr["account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [27]:
### user.remove()

user_id = "371ace7c-8abb-484e-9c61-ac1ab2247f4d"
with server.auth.sign_in(tableau_auth):
    server.users.remove(user_id)
    hr["account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [33]:
hr.to_csv("D:/디지털혁신팀/태블로/hr.csv", encoding="euc-kr")

## Projects

In [48]:
with server.auth.sign_in(tableau_auth):
    all_project_items, pagination_item = server.projects.get(req_options=req_opts)
    print([(prj.name, prj.id) for prj in all_project_items])

[('기본값', '7ff34228-4f13-11eb-8d94-cbbd9e107787'), ('Tableau 샘플', '07b1c1a3-d2fb-4a12-b4ba-c65b396f593c'), ('HDC현대산업개발', '2a21240d-3f53-4f02-944d-b01647c3dd51'), ('6월 데이터 확인', '44546321-94c8-4746-b0e5-0dfb3fa4ce5a')]


## Workbooks

In [307]:
# with server.auth.sign_in(tableau_auth):
#     all_wbs, pagination_item = server.workbooks.get()
# all_wbs = [(wb.project_name, wb.name) for wb in all_wbs]
# all_wbs = pd.DataFrame(all_wbs, columns=["prj", "wb"])
# all_wbs = all_wbs.sort_values(by=["prj", "wb"])

## Groups

## groups.get(), groups.populate_users()

In [113]:
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])
# all_groups = all_groups.sort_values(by=["group", "group_id"])
# all_groups = all_groups.reset_index(drop=True)

In [114]:
all_groups.to_csv("D:/디지털혁신팀/태블로/all_groups.csv", index=False, encoding="euc-kr")
all_groups.head()

,group,group_id
0,All Users,80307210-4f13-11eb-8d95-5bc411fd9c23
1,HR현황,8bae0a19-c83c-4713-b932-a489754653de
2,디지털혁신팀,516aa22e-b1ef-47b1-ad64-6e5116622c7f
3,본부장 이상,255400fe-a538-4b6e-80b9-63bb3c4362ae
4,분양일보,16065e45-21b3-4e1b-80e1-c7c41079adfd


In [82]:
group_user = list()
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    for group in groups[1:]:
        pagination_item = server.groups.populate_users(group, req_options=req_opts)
        group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

group_user = pd.DataFrame(group_user, columns=["group", "hq", "dep", "job", "name", "site_role"])
group_user = group_user.sort_values(by=["group", "hq", "dep", "job", "name", "site_role"])
group_user = group_user.reset_index(drop=True)

In [83]:
group_user.to_csv("D:/디지털혁신팀/태블로/group_user.csv", index=False, encoding="euc-kr")
group_user.head()

,group,hq,dep,job,name,site_role
0,HR현황,건설본부,건설기획팀,팀원,심창식,Viewer
1,HR현황,미래혁신본부,HR혁신팀,팀장,선영욱,Viewer
2,HR현황,미래혁신본부,디지털플랫폼팀,팀원,박미숙,Viewer
3,HR현황,미래혁신본부,미래전략팀,서무,김아현,Viewer
4,공수현황 접근그룹,개발영업본부,견적그룹(개발영업),그룹장,소문석,Viewer


In [24]:
user_name = "creator1"
site_role = "Creator"
newU = TSC.UserItem(user_name, site_role)

print(newU.name, newU.site_role, newU.workbooks)

UnpopulatedPropertyError: User item must be populated with workbooks first.

### groups.create()

In [66]:
try:
    with server.auth.sign_in(tableau_auth):
        newgroup = TSC.GroupItem("마이호미")
        newgroup.minimum_site_role = TSC.UserItem.Roles.Viewer
        newgroup = server.groups.create(newgroup)
except:
    pass

## Add Users to Groups

In [133]:
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    users, pagination_item = server.users.get(req_options=req_opts)
    for _, row in hr[hr["check"].notna()].iterrows():
        idx = all_groups[all_groups["group"] == row["check"]].index[0]
        group = groups[idx]
#         print(group)
        
        user_idx = all_users[all_users["user_name"] == row["num"]].index[0]
        user_id = users[user_idx].id
    
        server.groups.add_user(group, user_id)

In [76]:
keyword = "호미"
num = "5634"

group_idx = all_groups[all_groups["group"].str.contains(keyword)].index[0]
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    group = groups[group_idx]
    
    user_idx = all_users[all_users["user_name"] == num].index[0]
    users, pagination_item = server.users.get(req_options=req_opts)
    user_id = users[user_idx].id
    
    server.groups.add_user(group, user_id)

### users.populate_workbooks()

In [31]:
with server.auth.sign_in(tableau_auth):
    user = server.users.get_by_id("515c7d10-953c-4bfc-af9a-e87cf34ccb85")
    page_n = server.users.populate_workbooks(user)
    for workbook in user.workbooks :
        print(workbook.name)

분양일보_DM 테스트 (2)
AS_210324
마이호미_210326
menu navi test3
CS 현황_test
분양일보_test_210326
진행 현장 실적
Test (현장현황)
menu navi test2
Test (협업플랫폼 사용량)
현장 조직도_210319
마이호미_210329
분양일보_test_월 단위_210329
TEST
CS 현황_6월 데이터 확인
원자재 가격_210202
마이호미_210223
마이호미_210325
분양일보_DM 테스트
설비그룹 조직도_210322
분양일보_모바일
현장 현황test
menu navi
동호표_210331
CS 현황
홈 화면_디자인 변경
마이호미_210322
분양일보_test_210329
협업플랫폼 사용량
본부장회의자료_test
진행 현장 실적
본부장회의자료
마이호미


## Create Accounts

In [111]:
hr.head()

,num,hq,dep,sub,job,rank,name,check
1714,4100327,건설본부,경기남부고객만족센터,CS외주,A/S,외주직원2급,정기현,NaN
1671,4100193,건설본부,경기북부고객만족센터,CS외주,A/S,외주직원2급,최명찬,NaN
1676,4100247,건설본부,서울강북고객만족센터,CS외주,A/S,외주직원2급,한두식,NaN
1683,4100256,건설본부,서울강북고객만족센터,CS외주,A/S,외주직원2급,한강섭,NaN
1698,4100275,건설본부,경기남부고객만족센터,CS외주,A/S,외주직원3급,최윤용,NaN


In [115]:
with server.auth.sign_in(tableau_auth):
    name = "1101296"
    site_role = "Viewer"
    new_user = TSC.UserItem(name, site_role)
    
    new_user = server.users.add(new_user)   
    new_user.fullname = "황승환(디지털플랫폼팀)"
    new_user.email = "hwangsh@hdc-dvp.com"
    
    new_user = server.users.update(new_user, password=f"hdc{name}")

## Upload a Workbook

In [50]:
# filepath = "D:/디지털혁신팀/태블로/마이호미/마이호미_FINAL/마이호미.twbx"
# with server.auth.sign_in(tableau_auth):
#     wb = TSC.WorkbookItem(name="마이호미", project_id="44546321-94c8-4746-b0e5-0dfb3fa4ce5a")
#     wb = server.workbooks.publish(wb, filepath, "Overwrite")

### Sign Out

In [33]:
# server.auth.sign_out()